In [7]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import collections
import h5py
# import timm
import torch.nn as nn
import onnxruntime as ort
import torchaudio
import math

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from typing import Callable
from nnAudio.Spectrogram import CQT1992v2
# import leaf_audio_pytorch.frontend as frontend
from torchaudio.transforms import TimeMasking, FrequencyMasking
from shutil import copyfile
from torchaudio.transforms import Resample
from torchaudio.transforms import MelSpectrogram

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, load_pp_audio
# from code_base.utils.constants import SAMPLE_RATE, N_CLASSES_2021_2022
from code_base.utils.onnx_utils import convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier, WaveTDNNClasifier, RandomFiltering
from code_base.models.blocks import TraceableMelspec, NormalizeMelSpec, GeMFreqFixed, ChannelAgnosticAmplitudeToDB
# from code_base.models.wave_clasifier import WaveCNNAttenClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.datasets.wave_dataset import get_curate_id
from code_base.utils import load_json, write_json
from code_base.utils.audio_utils import get_librosa_load
from code_base.schedulers import GradualWarmupSchedulerV2
from code_base.utils.audio_utils import get_audio_metadata
# from code_base.augmentations.background_noise import AddBackgoundFromSoundScapes
# from code_base.augmentations.transforms import BackgroundNoise, Compose, GaussianNoise, PinkNoise, OneOf
# from code_base.forwards import MultilabelClsForwardLongShort
# from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.augmentations.transforms import (
    BackgroundNoise,
    Compose,
    GaussianNoise,
    OneOf,
    PinkNoise,
    TimeFlip
)
from code_base.utils.constants import ALL_AUDIO_FILTERS
from code_base.utils.main_utils import standardize_audio_filters

np.set_printoptions(precision=5, suppress=True)
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# parsed_filters = standardize_audio_filters(ALL_AUDIO_FILTERS)

In [ ]:
# parsed_filters

In [ ]:
dieter_noises = glob(
    "../data/dieter_background_noise/**/*.ogg", recursive=True
)
len(dieter_noises)

In [ ]:
ipd.Audio(filename=np.random.choice(dieter_noises))

In [ ]:
dieter_noises_meta = pd.DataFrame([get_audio_metadata(el) for el in tqdm(dieter_noises)])

In [ ]:
dieter_noises_meta["sample_rate"].value_counts()

In [ ]:
andrii_meta = pd.read_csv(
    "../data/add_train_audio_andrii_sample_22khz/splits.csv"
).drop(columns="Unnamed: 0")

In [ ]:
andrii_meta = pd.read_csv(
    "../data/birdclef_2025_crawl_dataset/archive/metadata.csv", delimiter=";"
)
# All other duplicates in "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa.csv"
andrii_meta = andrii_meta.iloc[:7]

andrii_meta

In [ ]:
andrii_meta = andrii_meta.iloc[:7]

In [ ]:
set(andrii_meta["primary_label"]) - set(df["primary_label"])

In [ ]:
# andrii_meta["primary_label"] = andrii_meta["ebird_code"]
andrii_meta["secondary_labels"] = [[] for _ in range(len(andrii_meta))]
# andrii_meta["filename"] = andrii_meta["short_filename"]
andrii_meta["data_root_id"] = "add_train_audio_crawl"
# andrii_meta["url"] = "andrii_url"

In [ ]:
add_audio_meta = pd.DataFrame(andrii_meta["filename"].apply(lambda x: get_audio_metadata(os.path.join(
    "../data/add_train_audio_crawl/",
    x
))).to_list())

In [ ]:
add_audio_meta

In [ ]:
andrii_meta = pd.concat([andrii_meta, add_audio_meta], axis=1)

In [ ]:
andrii_meta.isna().sum()

In [ ]:
andrii_meta.to_csv(
    "../data/birdclef_2025_crawl_dataset/archive/metadata_filtered_extended.csv", index=False
)

In [ ]:
andrii_meta.head()

In [ ]:
birds_vc = df["primary_label"].value_counts()

In [ ]:
birds_vc[birds_vc < 10].index

In [ ]:
birds_vc[birds_vc < 10]

In [ ]:
andrii_meta_uniform = andrii_meta[andrii_meta["uniform"]].reset_index(drop=True)
andrii_meta_filtered = andrii_meta[andrii_meta["filtered"]].reset_index(drop=True)
andrii_meta_filtered_limited = andrii_meta[andrii_meta["filtered_limited"]].reset_index(drop=True)
andrii_meta_filtered_oversampled = andrii_meta[andrii_meta["filtered_oversampled"]].reset_index(drop=True)

andrii_meta_uniform.to_csv(
    "../data/add_train_audio_andrii_sample_22khz/andrii_meta_uniform.csv", index=False
)
andrii_meta_filtered.to_csv(
    "../data/add_train_audio_andrii_sample_22khz/andrii_meta_filtered.csv", index=False
)
andrii_meta_filtered_limited.to_csv(
    "../data/add_train_audio_andrii_sample_22khz/andrii_meta_filtered_limited.csv", index=False
)
andrii_meta_filtered_oversampled.to_csv(
    "../data/add_train_audio_andrii_sample_22khz/andrii_meta_filtered_oversampled.csv", index=False
)

In [ ]:
andrii_meta_uniform.shape, andrii_meta_filtered.shape, andrii_meta_filtered_limited.shape, andrii_meta_filtered_oversampled.shape

In [ ]:
andrii_meta_filtered_oversampled = pd.read_csv(
    "../data/add_train_audio_andrii_sample_22khz/andrii_meta_filtered_oversampled.csv"
)

In [ ]:
andrii_meta_filtered_oversampled_vc = andrii_meta_filtered_oversampled["ebird_code"].value_counts()

In [ ]:
df_vc = df["primary_label"].value_counts()

In [ ]:
andrii_meta_filtered_oversampled_vc_dict = andrii_meta_filtered_oversampled_vc.to_dict()

In [ ]:
math.cei(andrii_meta_filtered_oversampled_vc_dict["turvul"] / df[df["primary_label"] == "turvul"].shape[0])

In [ ]:
math.ceil(0.3)

In [ ]:
def create_oversampled_df_for_class(
    original_df,
    class_name,
    n_add_samples
):
    class_sub_df = original_df[original_df["primary_label"] == class_name].reset_index(drop=True)
    repeat_times = math.ceil(n_add_samples / class_sub_df.shape[0])
    oversampled_df = pd.concat([class_sub_df] * repeat_times).sample(frac=1).reset_index(drop=True)
    return oversampled_df.iloc[:n_add_samples]

In [ ]:
turvul_add_df = create_oversampled_df_for_class(
    original_df=df,
    class_name="turvul",
    n_add_samples=andrii_meta_filtered_oversampled_vc_dict["turvul"]
)

In [ ]:
turvul_add_df.shape

In [ ]:
andrii_meta_filtered_oversampled_vc_dict

In [ ]:
pd.concat([df_vc.loc[andrii_meta_filtered_oversampled_vc.index], andrii_meta_filtered_oversampled_vc], axis=1)

In [ ]:
# all_submited_models = glob(
#     "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/kaggle_datasets/bird_clef_2025_models/*.zip"
# )
# len(all_submited_models)

In [ ]:
# accepted_regex = [
#     "Pseudo",
#     "FromV2Y2025Last",
# ]
# for sub_m in all_submited_models:
#     if not any(el in sub_m for el in accepted_regex):
#         os.remove(sub_m)
        

In [ ]:
chkp = torch.load(
    "/gpfs/helios/home/volodymyr1/src/bird_clef_2025/logdirs/convnext_small_fb_in22k_ft_in1k_384_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch50_SpecAugV1_FocalLoss_Full_PretrainPrevCompSp10/checkpoints/last.ckpt",
    map_location="cpu"
)

In [ ]:
from collections import OrderedDict

In [ ]:
chkp = chkp["state_dict"]

distilled_chkp = OrderedDict()
for key in chkp:
    if key.startswith("model.backbone."):
        distilled_chkp[key.replace("model.backbone.", "")] = chkp[key]

In [ ]:
TRAIN_PERIOD = 5.0
full_config = dict(
    backbone="convnext_small.fb_in22k_ft_in1k_384",
    mel_spec_paramms={
        "sample_rate": 32000,
        "n_mels": 128,
        "f_min": 20,
        "n_fft": 2048,
        "hop_length": 512,
        "normalized": True,
    },
    spec_augment_config={
        "freq_mask": {
            "mask_max_length": 10,
            "mask_max_masks": 3,
            "p": 0.3,
            "inplace": True,
        },
        "time_mask": {
            "mask_max_length": 20,
            "mask_max_masks": 3,
            "p": 0.3,
            "inplace": True,
        },
    },
    head_config={
        "p": 0.5,
        "num_class": 188,
        "train_period": TRAIN_PERIOD,
        "infer_period": TRAIN_PERIOD,
    },
    exportable=True,
)

In [ ]:
ful_model = WaveCNNAttenClasifier(**full_config,device="cpu")

In [ ]:
ful_model.backbone.load_state_dict(distilled_chkp)

In [ ]:
ful_model.backbone.state_dict().keys()

In [ ]:
backbone = timm.create_model(
    "convnext_small.fb_in22k_ft_in1k_384",
    features_only=True,
    pretrained=False,
    exportable=True,
    in_chans=1,
)

In [ ]:
backbone.load_state_dict(distilled_chkp)

In [ ]:
distilled_chkp.keys()

In [ ]:
temp_df = pd.read_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5.csv"
)

In [ ]:
temp_df["data_root_id"].value_counts()

In [ ]:
!ls -lt /home/vova/data/exps/birdclef_2024/kaggle_datasets/bird_clef_2024_addones/pretrained_backbones/

In [ ]:
one_df = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV3_meta_prev_comps_extended.csv")
second_df = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_NO2024_classes/train_metadata_noduplV5_extended_india_vc50.csv")

In [ ]:
one_df.shape[0] + second_df.shape[0]

In [ ]:
chkp = torch.load(
    "/home/vova/src/exps/bird_clef_2024/logdirs/eca_nfnet_l0_Exp_noamp_FixedAmp2DbAmin1e6_64bs_RandFirst10sec_AllPrevCompDataAndXcIndiaVC50_TimeFlip05_FormixupAlpha05NormedBinTgtEqW_balSampl_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_FullTrain_NoDuplsV5/checkpoints/last.ckpt", map_location="cpu"
)

In [ ]:
chkp = chkp["state_dict"]
chkp = {k.replace("model.backbone.", ""):v for k,v in chkp.items() if k.startswith("model.backbone.")}

In [ ]:
any([torch.isnan(tens).any() for tens in chkp.values()])

In [ ]:
chkp = torch.load(
    "/home/vova/data/exps/birdclef_2024/kaggle_datasets/bird_clef_2024_addones/pretrained_backbones/tf_efficientnetv2_b1_in1k_Pretrainversion2.pth", map_location="cpu"
)
any([torch.isnan(tens).any() for tens in chkp.values()])

In [ ]:
any([torch.isnan(tens).any() for tens in chkp.values()])

In [ ]:
32_000 * 5 // (256 - 1)

In [ ]:
def parameter_counter(input_model, model_name, only_trainable=False):
    counter = 0
    for param in input_model.parameters():
        if not only_trainable or param.requires_grad:
            counter += param.shape.numel()
    print(f"{model_name} Parameters Count: {f'{counter/1_000_000:.1f}M'}")

model1 = timm.create_model(
    "tf_efficientnetv2_b2.in1k",
    features_only=True,
    pretrained=True,
    exportable=True,
    in_chans=1,
)
parameter_counter(model1, "tf_efficientnetv2_b2_in1k")
model2 = timm.create_model(
    "tf_efficientnetv2_s",
    features_only=True,
    pretrained=True,
    exportable=True,
    in_chans=1,
)
parameter_counter(model2, "tf_efficientnetv2_s")

In [ ]:
model1 = timm.create_model(
    "tf_efficientnetv2_s_in21k",
    features_only=True,
    pretrained=True,
    exportable=True,
    in_chans=1,
)

In [ ]:
parameter_counter(model1, "tf_efficientnetv2_s_in21k")

In [ ]:
model2 = timm.create_model(
    "tf_efficientnetv2_s",
    features_only=True,
    pretrained=True,
    exportable=True,
    in_chans=1,
)
model2_sd = model2.state_dict()

In [ ]:
parameter_counter(model2, "tf_efficientnetv2_s")

In [ ]:
all(torch.allclose(model1_sd[key], model2_sd[key]) for key in model1_sd.keys())

In [ ]:
model.load_state_dict(chkp)

In [ ]:
torch.save(
    chkp,
    "/home/vova/data/exps/birdclef_2024/kaggle_datasets/bird_clef_2024_addones/pretrained_backbones/eca_nfnet_l0_Pretrainversion3.pth"
)

In [ ]:
chkp

In [ ]:
len(glob("/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes_features/*.hdf5"))

# Make Fast Holdout

In [ ]:
full_df = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/birdclef_2024/merged_train_metadata_extended_noduplv2.csv"
)
train_data = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv"
)

In [ ]:
def extract_id(fname):
    fname = os.path.splitext(os.path.basename(fname))[0]
    if fname.startswith("XC"):
        fname = fname[2:]
    return int(fname)

In [ ]:
train_data["id"] = train_data["filename"].apply(extract_id)
full_df["id"] = full_df["filename"].apply(extract_id)

In [ ]:
full_df["id"].isin(train_data["id"]).sum()

In [ ]:
np.where(~full_df["id"].isin(train_data["id"]))[0]

In [ ]:
cv_split = [
    [
        np.where(full_df["id"].isin(train_data["id"]))[0],
        np.where(~full_df["id"].isin(train_data["id"]))[0]
    ]
]

In [ ]:
np.save(
    "/home/vova/data/exps/birdclef_2024/cv_splits/merged_holdout.npy",
    np.array(cv_split, dtype=object)
)

In [ ]:
len(glob("/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes_features/*.hdf5"))

# Data

In [ ]:
def sample_uniformly_np_index(merged_intervals):
    # Calculate the total span of all intervals in terms of seconds
    total_seconds = sum(interval[1] - interval[0] for interval in merged_intervals)

    # Generate a random position within the total span as a float
    random_position = np.random.uniform(0, total_seconds)
    
    # Map the random position to an actual interval
    current_position = 0
    for index, (start, end, labels) in enumerate(merged_intervals):
        interval_length = end - start
        if current_position + interval_length > random_position:
            # Calculate the actual second within the interval as a float
            sampled_second = start + (random_position - current_position)
            return sampled_second, index
        current_position += interval_length

    raise RuntimeError("sample_uniformly_np_index was not able to pick chunk")

In [8]:
# df = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv")
# df = pd.read_csv(
#     "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_fixedaudiometa_h5pyDur.csv"
# )

df = pd.read_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_inat_fixedaudiometa.csv"
)

old_df = pd.read_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa_h5pyDur.csv"
)
cv_split = np.load(
    "../data/cv_split_base_and_prev_comps_XCsnipet28032025_group_allrarebirds_hdf5_noleak.npy", allow_pickle=True
)
# df = pd.read_csv(
#     "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1_GBirdV1.csv",
#     # converters={"label_second": eval}
# )
# df = pd.read_csv(
#     "/home/vova/src/exps/bird_clef_2024/google_bird_predictions/unlabeled_data/processed_data_v1_add_train.csv",
#     # converters={"label_second": eval}
# )
# taxonomy = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/eBird_Taxonomy_v2021.csv")
# previous_soundscapes = pd.read_csv(
#     "/home/vova/data/exps/birdclef_2024/my_2023_data/soundscapes_nocall/v1_no_call_meta_fixed.csv"
# )
# scored_birds = load_json(
#     "/home/vova/data/exps/birdclef_2024/scored_birds/sb_2024.json"
# )
# df = pd.read_csv("/home/vova/data/exps/BirdCLEF_2023/xeno_canto/train_metadata_extended_shorten_and_2023x_v1.csv")
# df.head()

# df["sample_id"] = df["filename"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

/tmp/ipykernel_3274531/4220632548.py:6: DtypeWarning: Columns (5,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [5]:
old_df["collection"].value_counts()

collection
XC      28670
iNat     7198
CSA       162
Name: count, dtype: int64

In [3]:
old_df["data_root_id"].value_counts()

data_root_id
train_audio                                 28564
add_train_audio_from_xeno_canto_28032025     7376
add_train_audio_from_prev_comps                90
Name: count, dtype: int64

In [6]:
old_df["author"]

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,...,license,sample_rate,duration_s,num_channels,bit_depth,encoding,id,dataset,data_root_id,duration_s_h5py
0,1139490,[],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,...,cc-by-nc-sa 4.0,32000,98.882000,1,0,VORBIS,36385,comp_2025,train_audio,98.853375
1,1139490,[],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,...,cc-by-nc-sa 4.0,32000,96.562000,1,0,VORBIS,36389,comp_2025,train_audio,96.537719
2,1192948,[],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,...,cc-by-nc-sa 4.0,32000,116.602000,1,0,VORBIS,36358,comp_2025,train_audio,116.599812
3,1192948,[],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,...,cc-by-nc-sa 4.0,32000,105.466000,1,0,VORBIS,36366,comp_2025,train_audio,105.446313
4,1192948,[],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,...,cc-by-nc-sa 4.0,32000,103.650000,1,0,VORBIS,36373,comp_2025,train_audio,103.631469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36025,greani1,[],['call'],greani1/108016.mp3,XC,NaN,https://xeno-canto.org/108016,-27.5059,-58.2139,Crotophaga major,...,//creativecommons.org/licenses/by-nc-nd/2.5/,44100,17.867755,2,0,MP3,108016,xeno_canto_only_2025,add_train_audio_from_xeno_canto_28032025,17.867781
36026,greani1,"['butsal1', 'socfly1', 'grekis', 'pavpig2']","['call', 'chorus call']",greani1/95618.mp3,XC,NaN,https://xeno-canto.org/95618,-0.4714,-76.4603,Crotophaga major,...,//creativecommons.org/licenses/by-nc-nd/2.5/,44100,42.031020,1,0,MP3,95618,xeno_canto_only_2025,add_train_audio_from_xeno_canto_28032025,42.031031
36027,greani1,[],['song'],greani1/86192.mp3,XC,NaN,https://xeno-canto.org/86192,9.5050,-75.3690,Crotophaga major,...,//creativecommons.org/licenses/by-nc-nd/2.5/,44100,73.822041,1,0,MP3,86192,xeno_canto_only_2025,add_train_audio_from_xeno_canto_28032025,73.822063
36028,greani1,[],['chorus'],greani1/8535.mp3,XC,NaN,https://xeno-canto.org/8535,-3.8260,-73.3760,Crotophaga major,...,//creativecommons.org/licenses/by-nc-nd/2.5/,44100,16.459093,1,0,MP3,8535,xeno_canto_only_2025,add_train_audio_from_xeno_canto_28032025,16.459094


In [ ]:
pseudo = pd.read_csv(
    "../data/bird_clef_2025_predictions/PseudI2_eca_l0_tf_efficientnetv2_s__nodupl.csv"
)

In [ ]:
pseudo["sample_id"] = pseudo["row_id"].apply(lambda x: "_".join(x.split("_")[:-1]))
pseudo["end_second"] = pseudo["row_id"].apply(lambda x: int(x.split("_")[-1]))

In [ ]:
label_str2int = load_json(
    "../data/bird2int_2025.json"
)

label_int2str = {v: k for k, v in label_str2int.items()}
arranged_birds = [label_int2str[idx] for idx in range(len(label_int2str))]

In [ ]:
sample2oofprob = (
    pseudo.groupby("sample_id")
    .apply(lambda sample_id_df: sample_id_df.sort_values("end_second")[arranged_birds].values)
    .to_dict()
)

In [ ]:
# sample2oofprob["CSA18794"]

In [ ]:
pseudo

In [ ]:
df["sample_rate"].value_counts()

In [ ]:
df["data_root_id"].value_counts()

In [ ]:
pl_vc_old = df.loc[~df["data_root_id"].isin(["csa_train_audio_parsed_11052025", "add_train_audio_from_xeno_canto_11052025"]), "primary_label"].value_counts()

In [ ]:
shorten_add_df = df[(       
    df["data_root_id"].isin(["csa_train_audio_parsed_11052025", "add_train_audio_from_xeno_canto_11052025"]) &
        df["primary_label"].isin(pl_vc_old[pl_vc_old < 10].index)
    )
].reset_index(drop=True)

In [ ]:
original_df = df[~df["data_root_id"].isin(["csa_train_audio_parsed_11052025", "add_train_audio_from_xeno_canto_11052025"])].reset_index(drop=True)

In [ ]:
original_df.shape[0], old_df.shape

In [ ]:
old_df_shorten = pd.concat([original_df, shorten_add_df]).reset_index(drop=True)

In [ ]:
old_df_shorten.to_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_fixedaudiometa_h5pyDur_shortenonly10classes.csv",
    index=False
)

In [ ]:
old_df_shorten.shape

In [ ]:
df[
    (
        df["data_root_id"].isin(["csa_train_audio_parsed_11052025", "add_train_audio_from_xeno_canto_11052025"]) &
        df["primary_label"].isin(pl_vc_old[pl_vc_old < 10].index)
    )
]

In [ ]:
pl_vc_old = df.loc[~df["data_root_id"].isin(["csa_train_audio_parsed_11052025", "add_train_audio_from_xeno_canto_11052025"]), "primary_label"].value_counts()

In [ ]:
pl_vc_old[pl_vc_old < 10].index

In [ ]:
df["da"]

In [ ]:
df["data_root_id"].value_counts()

In [ ]:
old_df["data_root_id"].value_counts()

In [ ]:
(df.loc[
    df["data_root_id"].isin(old_df["data_root_id"]),
    "filename"
] == old_df["filename"]).all()

In [ ]:
df["data_root_id"].value_counts()

In [ ]:
bird_vc = df["primary_label"].value_counts()
rare_birds = bird_vc[bird_vc <= 10].index
rare_ids = set(df.loc[df["primary_label"].isin(rare_birds), "filename"])

In [ ]:
for train_ids, val_ids in cv_split:
    assert df.iloc[train_ids]["filename"].value_counts().max() == 1
    assert set(df.iloc[train_ids]["filename"]) >= rare_ids
    assert not (set(df.iloc[train_ids]["filename"]) & set(df.iloc[val_ids]["filename"]))

In [ ]:
df["filename"].value_counts()

In [ ]:
assert set(df.iloc[train_ids]["filename"]) >= rare_ids 

In [ ]:
oof_df = pd.read_csv(
    "../data/bird_clef_2025_predictions/PseudI2_eca_l0_tf_efficientnetv2_s__nodupl.csv"
)

In [ ]:
oof_df.iloc[:,1:-1].values.min()

In [ ]:
oof_df["sample_id"] = oof_df["row_id"].apply(lambda x: "_".join(x.split("_")[:-1])) 
oof_df["end_second"] = oof_df["row_id"].apply(lambda x: int(x.split("_")[-1])) 

In [ ]:
label_str2int = load_json("../data/bird2int_2025.json")
label_int2str = {v:k for k,v in label_str2int.items()}

In [ ]:
arranged_birds = [label_int2str[idx] for idx in range(len(label_int2str))]

In [ ]:
sample2prob = oof_df.groupby("sample_id").apply(lambda sample_id_df: sample_id_df.sort_values("end_second")[arranged_birds].values).to_dict()

In [ ]:
sample2prob["791519"].shape

In [ ]:
df["sample_id"] = df["filename"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

In [ ]:
df[df["sample_id"] == "791519"]

In [ ]:
sample2prob["XC245490"].shape

In [ ]:
oof_df["sample_id"].value_counts()

In [ ]:
sub_df = oof_df[oof_df["sample_id"] == "XC245490"].iloc[:10, 1:10]

In [ ]:
oof_df[oof_df["sample_id"] == "XC245490"].iloc[:10]

In [ ]:
sub_df

In [ ]:
get_weighted_probability_vector(
    44,
    sub_df.values
)

In [ ]:
sub_df.iloc[-2].values * (1/5) + sub_df.iloc[-1].values * (4/5)

In [ ]:
def get_weighted_probability_vector(start_second, pseudo_probs, chunk_duration=5, rounding_precision=3):
    first_chunk_id = int(start_second // chunk_duration)
    first_chunk_duration = ((first_chunk_id + 1) * chunk_duration) - start_second
    assert first_chunk_duration <= chunk_duration
    first_chunk_weight = first_chunk_duration / chunk_duration
    if (first_chunk_id + 1) == pseudo_probs.shape[0]:
        assert round(first_chunk_duration, rounding_precision) == chunk_duration
        return pseudo_probs[first_chunk_id]
    elif (first_chunk_id + 1) > pseudo_probs.shape[0]:
        raise RuntimeError("Run out of pseudo_probs length")
    else:
        return (
            pseudo_probs[first_chunk_id] * first_chunk_weight + 
            pseudo_probs[first_chunk_id + 1] * (1 - first_chunk_weight)
        )

In [ ]:
df["label_second"].iloc[13]

In [ ]:
sample_uniformly_np_index(df["label_second"].iloc[13])

In [ ]:
seconds = [sample_uniformly_np_float(df["label_second"].iloc[13])[0] for _ in range(100000)]

In [ ]:
plt.hist(seconds, bins=100);

In [ ]:
# previous_soundscapes["filename"] = previous_soundscapes["filename"].apply(lambda x: os.path.join(
#     "/home/vova/data/exps/birdclef_2024/my_2023_data/soundscapes_nocall/train_audio", x
# ))

# def check_is_capable_to_load(path):
#     try:
#         librosa.load(path, sr=32000)
#         return True
#     except:
#         return False

# previous_soundscapes["is_corrupted"] = previous_soundscapes["filename"].apply(check_is_capable_to_load)

# or_previous_soundscapes = pd.read_csv(
#     "/home/vova/data/exps/birdclef_2024/my_2023_data/soundscapes_nocall/v1_no_call_meta.csv"
# )
# or_previous_soundscapes[previous_soundscapes["is_corrupted"]].reset_index(drop=True).to_csv(
#     "/home/vova/data/exps/birdclef_2024/my_2023_data/soundscapes_nocall/v1_no_call_meta_fixed.csv",
#     index=False
# )

In [ ]:
# previous_soundscapes["primary_label"] = "nocall"
# previous_soundscapes.to_csv(
#     "/home/vova/data/exps/birdclef_2024/my_2023_data/soundscapes_nocall/v1_no_call_meta.csv", index=False
# )

In [ ]:
df

In [ ]:
df.shape[0] / 64

In [ ]:
df["secondary_labels"].apply(eval).apply(len).value_counts()

In [ ]:
df

In [ ]:
df = df.sample(frac=1).reset_index(drop=True).iloc[:100]


In [ ]:
df["primary_label"].value_counts()

In [ ]:
df["duration_s"].iloc[:10].max()

In [ ]:
ab = 0.75
plt.hist(np.clip(np.random.beta(ab, ab, size=1000), 0.2, 0.8))

In [ ]:
mel_spec_paramms={
    "sample_rate": 32000,
    "n_mels": 128,
    "f_min": 20,
    "n_fft": 2048,
    "hop_length": 512,
    "normalized": True,
}
mel_spec = nn.Sequential(
    TraceableMelspec(**mel_spec_paramms, quantizable=False),
    ChannelAgnosticAmplitudeToDB(top_db=80.0),
    NormalizeMelSpec(exportable=True),
)

In [ ]:
background_module = BackgroundNoise(**{
    "esc50_root": "/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes/",
    "esc50_df_path": "../pseudo/v1/negative_filtered_02.csv",
    "precompute": False,
    "load_normalize": False,
    "normalize_chunks": True,
    "normalize": True,
    "chunks_column": "nocall"
})

In [ ]:
audio = background_module.sample(32000 * 5)
spec = mel_spec(torch.from_numpy(audio)[None,...])
plt.imshow(spec[0])
ipd.Audio(audio, rate=32000)

In [ ]:
audio.shape[0] / 32000

In [ ]:
background_module.sample_names[1482]

In [ ]:
np.allclose(
    librosa.util.normalize(
        librosa.load(background_module.sample_names[1482], sr=32_000)[0][:5*32000]
    ), 
    audio
)

In [ ]:
# np.abs(background_module.sample(32000*5)).max()

In [ ]:
# np.abs(background_module._pick_random_valid_sample()).max()

In [ ]:
plt.hist(np.random.beta(0.5, 0.5, size=(10000)))

In [ ]:
"curation_json_path": "data/train_data_curation_10042025_sr32K.json"

In [10]:
train_dataset = WaveDataset(
    df=old_df,
    root="../data/train_audio",
    label_str2int_mapping_path="../data/bird2int_2025.json",
    precompute=False,
    n_cores=4,
    debug=False,
    segment_len=5,
    late_normalize=True,
    sampler_col="primary_label",
    use_h5py=True,
    replace_pathes=("train_audio", "train_features"),
    filename_change_mapping={
        "base": "train_audio",
        "train_audio": "train_audio",
        "add_train_audio_from_prev_comps": "add_train_audio_from_prev_comps",
        "add_train_audio_from_xeno_canto_28032025": "add_train_audio_from_xeno_canto_28032025",
        "csa_train_audio_parsed_11052025": "csa_train_audio_parsed_11052025",
        "add_train_audio_from_xeno_canto_11052025": "add_train_audio_from_xeno_canto_11052025",
        "soundscape_0": "train_features_soundscapes",
        "soundscape_1": "train_features_soundscapes",
        "soundscape_2": "train_features_soundscapes",
    },
    ignore_setting_dataset_value=True,
    label_smoothing=0.05,
    # curation_chunks_json_path="../data/curation_v3.json"
    # use_label_smoothing_fix=True,
    soundscape_pseudo_df_path=[
        "../data/pseudo/eca_124__eca_117__eca_112__eca_118__eca_113__ebs_123__eca_120__eca_121/v0_0.csv",
        "../data/pseudo/double_eca_nfnet_l0_AllFPseudoI1/v0_0.csv",
        "../data/pseudo/best_ensem_19052025/v0.csv"
    ],
    soundscape_pseudo_config={
        "primary_label_min_prob": 0.5,#[0.5, 0.4],
        "trim_min_prob": 0.1,
        "sampling_prob": 0.4
    },
    # oof_df_path="../data/bird_clef_2025_predictions/PseudI2_eca_l0_tf_efficientnetv2_s__nodupl.csv",
    # oof_sampling_config={
    #     "trim_min_prob": 0.1,
    #     "sampling_prob": 1.0,
    #     "acceptance_prob": 0.5,
    #     "use_zero_samples": True,
    #     "zero_sample_prob": 0.3
    # }
    # do_mixup=True,
    # mixup_params={"prob": 0.5, "alpha": None},
    # label_smoothing=0.05
    # curation_json_path="../data/train_data_curation_10042025_sr32K.json"
)

# train_dataset_cur = WaveDataset(
#     df=df,
#     root="../data/train_audio",
#     label_str2int_mapping_path="../data/bird2int_2025.json",
#     precompute=False,
#     n_cores=4,
#     debug=False,
#     segment_len=5,
#     late_normalize=True,
#     sampler_col="primary_label",
#     duration_col="duration_s_h5py",
#     use_h5py=True,
#     replace_pathes=("train_audio", "train_features"),
#     filename_change_mapping={
#         "base": "train_audio",
#         "train_audio": "train_audio",
#         "add_train_audio_from_prev_comps": "add_train_audio_from_prev_comps",
#         "add_train_audio_from_xeno_canto_28032025": "add_train_audio_from_xeno_canto_28032025",
#         "soundscape_0": "train_features_soundscapes",
#         "soundscape_1": "train_features_soundscapes",
#     },
#     ignore_setting_dataset_value=True,
#     label_smoothing=0.05,
#     curation_json_path="../data/curation_v2.json",
#     curation_chunks=ALL_AUDIO_FILTERS
#     # use_label_smoothing_fix=True,
#     # soundscape_pseudo_df_path=[
#     #     "../data/pseudo/eca_124__eca_117__eca_112__eca_118__eca_113__ebs_123__eca_120__eca_121/v0_0.csv",
#     #     "../data/pseudo/double_eca_nfnet_l0_AllFPseudoI1/v0_0.csv"
#     # ],
#     # soundscape_pseudo_config={
#     #     "primary_label_min_prob": 0.5,#[0.5, 0.4],
#     #     "trim_min_prob": 0.1,
#     #     "sampling_prob": 0.4
#     # },
#     # oof_df_path="../data/bird_clef_2025_predictions/PseudI2_eca_l0_tf_efficientnetv2_s__nodupl.csv",
#     # oof_sampling_config={
#     #     "trim_min_prob": 0.1,
#     #     "sampling_prob": 0.4
#     # }
#     # do_mixup=True,
#     # mixup_params={"prob": 0.5, "alpha": None},
#     # label_smoothing=0.05
#     # curation_json_path="../data/train_data_curation_10042025_sr32K.json"
# )

Validating that all files exist


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 386166/386166 [00:00<00:00, 660356.34it/s]


Selected 4430 samples from 9726 from ../data/pseudo/eca_124__eca_117__eca_112__eca_118__eca_113__ebs_123__eca_120__eca_121/v0_0.csv
Selected 19405 rows
Selected 1483 samples from 9726 from ../data/pseudo/double_eca_nfnet_l0_AllFPseudoI1/v0_0.csv
Selected 3750 rows
Selected 1437 samples from 9726 from ../data/pseudo/best_ensem_19052025/v0.csv
Selected 4108 rows


In [ ]:
train_dataset.curation_chunks

In [ ]:
set(train_dataset.curation_chunks.keys()) - set(train_dataset.df["original_name_col"])

In [ ]:
curation_v3 = load_json(
    "../data/curation_v3.json"
)

In [ ]:
[key for key, value in curation_v3.items() if value[0][0] == -1]

In [ ]:
curation_v3["grepot1/114330.wav"]

In [ ]:
train_dataset.df["data_root_id"].value_counts()

In [ ]:
df[df["filename"] == "grepot1/114330.wav"]

In [ ]:
0.544594 * 1000

In [ ]:
df[df["primary_label"] == "whfant1"]

In [ ]:
df.iloc[37799]

In [ ]:
df["duration_s"].sort_values()

In [ ]:
train_dataset.df[
    (train_dataset.df["data_root_id"] == "csa_train_audio_parsed_11052025") &
    (train_dataset.df["id"] == 101191)
]

In [ ]:
def get_audio_fname(df_row):
    path = df_row["filename_with_root"]
    path = "/".join((*path.split("/")[:-2], df_row["filename"]))
    return path.replace("features", "audio")

In [ ]:
vad_model, (get_speech_timestamps, _, _, _, _) = torch.hub.load(
        repo_or_dir="snakers4/silero-vad", model="silero_vad"
    )
# if device == "cuda":
#     vad_model = vad_model.to(device)
vad_model.eval();

In [ ]:
audio, sr = torchaudio.load(get_audio_fname(train_dataset.df.iloc[19795]))
if sr != 32000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=32000)
    audio = resampler(audio)
    sr = 32000

In [ ]:
ipd.Audio(audio[0], rate=32000)

In [ ]:
len(audio[0]) / 32000

In [ ]:
def get_non_speech_regions(audio, sr, vad_model, get_speech_timestamps, device, threshold=0.5, min_speech_duration_ms=250, min_silence_duration_ms=1000):
    """
    Returns a list of (start, end) tuples (in seconds) for non-speech regions in the audio.
    Speech segments shorter than min_speech_len are ignored (not considered as speech).
    """
    if len(audio.shape) > 1:
        audio = audio[0]
    if not hasattr(audio, "to"):
        audio = torch.tensor(audio)
    audio = audio.to(device)
    total_len = audio.shape[-1]
    audio_ms = total_len / sr * 1000
    if audio_ms < min_silence_duration_ms:
        min_silence_duration_ms = 100
    speech_timestamps = get_speech_timestamps(
        audio, 
        vad_model, 
        sampling_rate=sr, 
        threshold=threshold,
        min_speech_duration_ms=min_speech_duration_ms,
        min_silence_duration_ms=min_silence_duration_ms
    )
    total_len = audio.shape[-1]
    non_speech = []
    prev_end = 0
    for ts in speech_timestamps:
        start = ts["start"]
        end = ts["end"]
        # Only add non-speech region if its duration is >= min_silence_duration_ms
        # chunk_ms = (start - prev_end) / sr * 1000
        # if start > prev_end and chunk_ms >= min_silence_duration_ms:
        if start > prev_end:
            non_speech.append((prev_end / sr, start / sr))
        prev_end = end
    # Check last region
    # chunk_ms = (total_len - prev_end) / sr * 1000
    # if prev_end < total_len and chunk_ms >= min_silence_duration_ms:
    if prev_end < total_len:
        non_speech.append((prev_end / sr, total_len / sr))
    return non_speech

In [ ]:
len(audio[0]) / 32000, len(torch.cat([audio, torch.zeros(*(1,int((32000 / 1000) *2.1))).float()], dim=-1)[0]) / 32000

In [ ]:
get_non_speech_regions(
    audio=audio,
    sr=32000,
    vad_model=vad_model,
    get_speech_timestamps=get_speech_timestamps,
    device="cpu"
)

In [ ]:
np.array([0.1, 0.33, 0.66, 0.33]) / sum(np.array([0.1, 0.33, 0.66, 0.33])) 

In [ ]:
speech_timestamps = get_speech_timestamps(audio[0], vad_model, sampling_rate=sr, threshold=0.5, min_speech_duration_ms=250,min_silence_duration_ms=1000 * 2)

In [ ]:
[{"start": el["start"] / 32000, "end": el["end"] / 32000} for el in speech_timestamps]

In [ ]:
get_audio_fname(train_dataset.df.iloc[36606])

In [ ]:
214400 / 32000

In [ ]:
curation_json = load_json(
    "../data/curation_v2.json"
)

In [ ]:
next(iter(curation_json.values()))

In [ ]:
old_df["curate_id"] = df["filename"].apply(get_curate_id)

In [ ]:
old_df["curate_end"] = old_df.apply(
    lambda row: curation_json[row["curate_id"]][1] / 32000 if curation_json[row["curate_id"]][1] != -1 else row["duration_s"],
    axis=1
)

In [ ]:
old_df[
    (np.abs(old_df["curate_end"] - old_df["duration_s"]) > 3) & 
    (old_df["collection"] == "XC")
]

In [ ]:
df[df["filename"].apply(get_curate_id) == "compot1/104826"]

In [ ]:
curation_json["grepot1/XC403324"]

In [ ]:
min(v[1] - v[0] for v in curation_json.values() if v[0] != -1)

In [ ]:
{
    k:(v[1] - v[0] if v[0] != -1 else -1) for k,v in curation_json.items() if (v[1] - v[0] == 0)
}

In [ ]:
train_dataset.df.shape, train_dataset_cur.df.shape

In [ ]:
train_dataset_cur.curation_chunks["gybmar/XC9608"]

In [ ]:
lengths / (lengths.sum() + 1e-6)

In [ ]:
np.random.choice(
    range(train_dataset_cur.curation_chunks["42007/iNat15105"].shape[0]),
    p=lengths / (lengths.sum() + 1e-6)
)

In [ ]:
{k:min(v[:,1] - v[:,0]) for k, v in train_dataset_cur.curation_chunks.items()}

In [ ]:
train_dataset.df["filename_with_root"].apply(os.path.exists).all()

In [ ]:
torchaudio_meta = pd.DataFrame(train_dataset.df["filename_with_root"].progress_apply(get_audio_metadata).to_list())

In [ ]:
librosa_length = train_dataset.df["filename_with_root"].progress_apply(lambda x: librosa.get_duration(filename=x))

In [ ]:
(librosa_length.apply(lambda x: round(x,1)) - torchaudio_meta["duration_s"].apply(lambda x: round(x,1)))

In [ ]:
(df["duration_s"].apply(lambda x: round(x, 3)) == librosa_length.apply(lambda x: round(x, 3))).all()

In [ ]:
df[torchaudio_meta.columns] = torchaudio_meta

In [ ]:
df[torchaudio_meta.columns].iloc[35378]

In [ ]:
# df.to_csv(
#     "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa.csv",
#     index=False
# )

In [ ]:
torchaudio_meta.iloc[35378]

In [ ]:
df["encoding"]

In [ ]:
torchaudio_meta["encoding"]

In [ ]:
torchaudio_meta["duration_s"].apply(lambda x: round(x,1))

In [ ]:
torchaudio_meta["sample_rate"].value_counts()

In [ ]:
train_dataset.df[train_dataset.df["sample_id"] == "791519"]

In [ ]:
andrii_meta_filtered_oversampled_vc_dict

In [ ]:
(train_dataset_andrii.df["primary_label"].value_counts() == train_dataset_oversample.df["primary_label"].value_counts()).all()

In [ ]:
train_dataset_oversample.df["primary_label"].value_counts()

In [ ]:
train_dataset_without_cur.df["data_root_id"].value_counts()

In [ ]:
train_dataset_without_cur.soundscape_df["data_root_id"].value_counts()

In [ ]:
train_dataset_without_cur.soundscape_df["primary_label"].value_counts()

In [ ]:
train_dataset_without_cur[np.random.randint(0, len(train_dataset_without_cur))]

In [ ]:
sample_idx = np.random.choice(np.where(train_dataset_without_cur.df["data_root_id"] == "add_train_audio_andrii_generated_undersample")[0])

In [ ]:
curation_json = load_json("../temp.json")

In [ ]:
train_dataset_cur.df["temp_id"] = train_dataset_cur.df["filename"].apply(lambda x: os.path.splitext(x)[0])

In [ ]:
train_dataset_cur.df[train_dataset_cur.df["temp_id"] == "gybmar/XC9608"]

In [ ]:
curation_json["41778/XC959831"][-1] / 32000

In [ ]:
curation_json["41778/XC959831"][0] / 32000

In [ ]:
[key for key, value in curation_json.items() if value[0] > 32000 * 10][:10]

In [ ]:
list(curation_json.keys())[:10]

In [ ]:
sample_idx = np.random.randint(len(train_dataset))#np.random.choice(np.where(train_dataset_without_cur.df["data_root_id"] == "add_train_audio_andrii_generated_undersample")[0])

au, target = train_dataset[sample_idx]

print(target.numpy())

In [ ]:
target.max()

In [ ]:
pd.Series(target).value_counts()

In [ ]:
train_dataset_cur.curation_chunks["41778/XC959831"] / 32000

In [ ]:
sample_idx = 38271#np.random.randint(len(train_dataset))#np.random.choice(np.where(train_dataset_without_cur.df["data_root_id"] == "add_train_audio_andrii_generated_undersample")[0])

au, target = train_dataset[sample_idx]

# print(target.numpy())

In [ ]:
ipd.Audio(data=au, rate=train_dataset.sample_rate)

In [ ]:
curation_v3["grepot1/114330.wav"]

In [ ]:
train_dataset_cur.curation_chunks["gybmar/XC9608"]

In [ ]:
target.max()

In [ ]:
temp_au, temp_sr = librosa.load(
    "../data/add_train_audio_from_xeno_canto_28032025/strher/791519.mp3",
    sr=22050
)

In [ ]:
temp_au.shape[0] / temp_sr

In [ ]:
temp_au.shape

In [ ]:
temp_sr

In [ ]:
get_audio_metadata("../data/add_train_audio_from_xeno_canto_28032025/strher/791519.mp3")

In [ ]:
librosa.get_duration(filename="../data/add_train_audio_from_xeno_canto_28032025/strher/791519.mp3")

In [ ]:
train_dataset.df["duration_s"]

In [ ]:
h5py_durations_in_seconds = [] 
for fname, count_dur in tqdm(zip(train_dataset.df["filename_with_root"], train_dataset.df["duration_s"])):
    with h5py.File(fname, "r", swmr=True) as f:
        real_dur = len(f["au"]) / 32000
        h5py_durations_in_seconds.append(real_dur)
        # if abs(round(count_dur, 3) - round(real_dur, 3)) > 2:
        #     print(fname, count_dur, real_dur)

In [ ]:
df["duration_s_h5py"] = h5py_durations_in_seconds

In [ ]:
df.apply(lambda row: os.path.join("../data", row["data_root_id"], row["filename"]), axis=1)

In [ ]:
audio_meta = pd.DataFrame(df.apply(lambda row: os.path.join("../data", row["data_root_id"], row["filename"]), axis=1).apply(get_audio_metadata).to_list())

In [ ]:
audio_meta

In [ ]:
df[audio_meta.columns] = audio_meta

In [ ]:
df.to_csv(
    "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_fixedaudiometa_h5pyDur.csv",
    index=False
)

In [ ]:
# df["primary_label"].value_counts()[df["primary_label"].value_counts() < 10]

In [ ]:
df["duration_s"] - df["duration_s_h5py"] 

In [ ]:
# df.to_csv(
#     "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5_fixedaudiometa_h5pyDur.csv",
#     index=False
# )

In [ ]:
temp_au, temp_sr = torchaudio.load("../data/add_train_audio_from_xeno_canto_28032025/strher/791519.mp3")

In [ ]:
temp_au.shape[-1] / temp_sr

In [ ]:
# resampler = Resample(
#     orig_freq=temp_sr, new_freq=32_000
# )
temp_au_32K = torchaudio.functional.resample(temp_au.unsqueeze(0), orig_freq=temp_sr, new_freq=32_000) #resampler.forward(temp_au)

In [ ]:
temp_au_32K.shape[-1] / 32000

In [ ]:
temp_au_32K.shape

In [ ]:
temp_au_32K

In [ ]:
plt.plot(temp_au_32K[0, -32000*10:])

In [ ]:
target.min(), target.max() 

In [ ]:
ipd.Audio(au, rate=22050)

In [ ]:
train_dataset_without_cur.df[train_dataset_without_cur.df["primary_label"].isin(train_dataset_without_cur.soundscape_df["primary_label"])]

In [ ]:
train_dataset_without_cur[1000][0].shape[0] / 32000

In [ ]:
boundaries = load_json(
    "../data/train_data_curation_10042025_sr32K.json"
)

In [ ]:
boundaries["blkvul/XC143654"][1] / 32000

In [ ]:
for key, value in boundaries.items():
    if value[-1] == -1:
        print(key)

In [ ]:
boundaries["21211/XC882652"]

In [ ]:
train_dataset.df["filename_with_root"].iloc[0]

In [ ]:
train_dataset.df["stratify_col"].isna().sum()

In [ ]:
not_base_df = train_dataset.df[train_dataset.df["dataset"] != "base"].reset_index(drop=True)

In [ ]:
for fname in tqdm(not_base_df["filename_with_root"].to_list()):
    try:
        copyfile(
            fname, 
            os.path.join("/home/vova/data/exps/birdclef_2024/kaggle_datasets/bird_clef_2024_add_data/scored_add_data", os.path.basename(fname))
        )
    except Exception as e:
        fname = fname.replace(".ogg", ".mp3")
        copyfile(
            fname, 
            os.path.join("/home/vova/data/exps/birdclef_2024/kaggle_datasets/bird_clef_2024_add_data/scored_add_data", os.path.basename(fname))
        )

In [ ]:
id2bird = {v:k for k,v in train_dataset.label_str2int.items()}

In [ ]:
len(set(train_dataset.df["primary_label"]))

In [ ]:
set(scored_birds) == set(train_dataset.df["primary_label"])

In [ ]:
train_dataset.df["dataset"].value_counts(normalize=True)

In [ ]:
train_dataset.df[train_dataset.df["dataset"] == "soundscape"]

In [ ]:
train_dataset.df[train_dataset.df.duration_s <= 5]

In [ ]:
train_dataset.df.duration_s.median()

In [ ]:
np.quantile(train_dataset.df.duration_s, 0.85)

In [ ]:
train_dataset.df.duration_s.hist(bins=30)

In [ ]:
train_dataset.df[train_dataset.df["dataset"] == "soundscape"]

In [ ]:
df.shape[0] / len(train_dataset), len(train_dataset)

In [ ]:
train_dataset[35530]

In [ ]:
len(train_dataset), len(train_dataset_ss), len(merged_dataset)

In [ ]:
pd.Series(train_dataset.targets).value_counts()

In [ ]:
all_tgts = pd.Series(train_dataset.targets)
sample_weights = (all_tgts.value_counts() / all_tgts.value_counts().sum())  ** (-0.5)
write_json(
    "/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_v1_pseudov1_StratV2.json",
    sample_weights.to_dict()
)

In [ ]:
# all_tgts = pd.Series(list(chain(*[ds.targets for ds in merged_dataset.datasets])))
# sample_weights = (all_tgts.value_counts() / all_tgts.value_counts().sum())  ** (-0.5)
# write_json(
#     "/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_add_data_v1_with_nocall.json",
#     sample_weights.to_dict()
# )

In [ ]:
class_weights = load_json("/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_add_data_v1_with_nocall.json")
sample_weights = np.array([
    class_weights[el] for el in list(chain(*[ds.targets for ds in merged_dataset.datasets]))
])
sampler = torch.utils.data.WeightedRandomSampler(sample_weights, len(sample_weights))

In [ ]:
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)

loader = torch.utils.data.DataLoader(
    merged_dataset,
    worker_init_fn=worker_init_fn,
    sampler=sampler,
    **{
        "batch_size": 64,
        "shuffle": False,
        "drop_last": True,
        "num_workers": 0,
        "pin_memory": True,
    },
)

In [ ]:
len(loader)

In [ ]:

for idx, b in enumerate(loader):
    print(f"Empty Fraction: {(b[1].sum(axis=1) == 0).sum() / b[1].shape[0]}")
    if idx > 10:
        break

In [ ]:
(batch[1].sum(axis=1) == 0).sum() / batch[1].shape[0]

In [ ]:
au, tgt = train_dataset_ss[0]

In [ ]:
np.abs(au).max(), tgt.sum()

In [ ]:
train_dataset_full[0][0].min()

In [ ]:
train_dataset_full.test_audio_cache["au"].min()

In [ ]:
sorted(set(train_dataset.df["primary_label"]))

In [ ]:
train_dataset.df.loc[train_dataset.df["dataset"] == "xc_2024_classes", "primary_label"].value_counts()

In [ ]:
taxonomy

In [ ]:
taxonomy[taxonomy["PRIMARY_COM_NAME"].apply(lambda x: x.startswith("Intermediate Egret"))]

In [ ]:
train_dataset.df[(train_dataset.df["dataset"] == "xc_2024_classes") & (train_dataset.df["primary_label"] == "integr")]

In [ ]:
# train_dataset.df[train_dataset.df.primary_label == "brnfio1"]

In [ ]:
train_dataset.df[(train_dataset.df["dataset"] == "base") & (train_dataset.df["primary_label"] == "integr")]

In [ ]:
train_dataset.label_str2int["asbfly"]

In [ ]:
torch.where(train_dataset[22][1] > 0)[0]

In [ ]:
train_dataset.df["filename_with_root"].apply(os.path.exists).all()

In [ ]:
train_dataset.df.shape

In [ ]:
vc = train_dataset.df["url"].apply(lambda x: x.split("/")[-1]).value_counts()

In [ ]:
train_dataset.df[]

In [ ]:
train_dataset.df[train_dataset.df["url"].apply(lambda x: x.split("/")[-1]).isin(vc[vc > 1].index)].sort_values("url")

In [ ]:
sample_au, target_au = train_dataset[1000]

In [ ]:
sample_au.min(), sample_au.max()

In [ ]:
df[df["primary_label"] == "comtai1"].url

In [ ]:
sample_weights = pd.Series(load_json("/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_add_data_v1.json"))

In [ ]:
sample_weights

In [ ]:
sampled_birds = []
for _ in tqdm(range(1000)):
    sample_au, target_au = train_dataset[0]
    sampled_birds.extend([id2bird[el] for el in np.where(target_au > 0)[0]])

In [ ]:
pd.Series(sampled_birds).value_counts().iloc[1:20]

In [ ]:
train_dataset.df.primary_label.value_counts().iloc[1:20]

In [ ]:
train_dataset.df["label_second"]

In [ ]:
train_dataset.df[train_dataset.df["label_second"].apply(lambda x: len(set(chain(*[el[-1] for el in x])))) > 1]

In [ ]:
train_dataset.df[
    (train_dataset.df["label_second"].apply(lambda x: x[-1][1] - x[-1][0]) < 2.5) &
    (train_dataset.df["label_second"].apply(len) == 1) &
    (train_dataset.df["duration_s"] > 5)
]

In [ ]:
train_dataset.df[train_dataset.df["duration_s"] < 5]

In [ ]:
int((27.35021875 - 5) * 32000)

In [ ]:
(33.64571875 - 5) * 32000

In [ ]:
(30 - 2.5) * 32000

In [ ]:
train_dataset.df[train_dataset.df.duration_s <= 5]

In [ ]:
mel_spec_paramms={
    "sample_rate": 32000,
    "n_mels": 128,
    "f_min": 20,
    "n_fft": 2048,
    "hop_length": 512,
    "normalized": True,
}
mel_spec = nn.Sequential(
    TraceableMelspec(**mel_spec_paramms, quantizable=False),
    AmplitudeToDB(top_db=80.0),
    NormalizeMelSpec(exportable=True),
)
mel_spec_new = nn.Sequential(
    TraceableMelspec(**mel_spec_paramms, quantizable=False),
    ChannelAgnosticAmplitudeToDB(top_db=80.0),
    NormalizeMelSpec(exportable=True),
)

In [ ]:
train_dataset.df.loc[train_dataset.df["dataset"] == "soundscape", "primary_label"].value_counts().head(20)

In [ ]:
train_dataset.df[
    (train_dataset.df["primary_label"] == "whcbar1") &
    (train_dataset.df["dataset"] != "soundscape")
]

In [ ]:
class_name = "brcful1"

In [ ]:
target_au

In [ ]:
# sample_au, target_au = train_dataset[0]
# sample_id = np.random.randint(0, len(train_dataset))
sample_id = 9831 #np.random.choice(train_dataset.df[
#     (train_dataset.df["primary_label"] == class_name) &
#     (train_dataset.df["dataset"] == "soundscape")
# ].index)
print("Sample ID", sample_id)
sample_au, target_au, sample_au_unlabeled = train_dataset[sample_id]

print("Target IDS:", np.where(target_au > 0)[0])
print("Birds: ", [id2bird[el] for el in np.where(target_au > 0)[0]])

# plt.figure(figsize=(20,10))
spec = mel_spec(torch.from_numpy((sample_au))[None,...]) 
plt.imshow(spec[0])
plt.show()

spec_unlabeled = mel_spec(torch.from_numpy((sample_au_unlabeled))[None,...]) 
plt.imshow(spec_unlabeled[0])
plt.show()

ipd.Audio(sample_au, rate=32000)

In [ ]:
ipd.Audio(sample_au_unlabeled, rate=32000)

In [ ]:
plt.hist(spec.flatten(), bins=30);

In [ ]:
160000.0

In [ ]:
# sample_au, target_au = train_dataset[0]
# sample_id = np.random.randint(0, len(train_dataset))
sample_id = 9831 #np.random.choice(train_dataset.df[
#     (train_dataset.df["primary_label"] == class_name) &
#     (train_dataset.df["dataset"] == "soundscape")
# ].index)
print("Sample ID", sample_id)
sample_au, target_au, mask = train_dataset[sample_id]

print(mask)

print("Target IDS:", np.where(target_au > 0)[0])
print("Birds: ", [id2bird[el] for el in np.where(target_au > 0)[0]])

# plt.figure(figsize=(20,10))
spec = mel_spec(torch.from_numpy((sample_au))[None,...]) 
plt.imshow(spec[0])
plt.show()

ipd.Audio(sample_au, rate=32000)

In [ ]:
torch.all(target_au == mask)

In [ ]:
# sample_au, target_au = train_dataset[0]
sample_id = 3770 #np.random.choice(train_dataset.df[
#     (train_dataset.df["primary_label"] == class_name) &
#     (train_dataset.df["dataset"] != "soundscape")
# ].index)
print("Sample ID", sample_id)
sample_au, target_au = train_dataset[sample_id]

print("Target IDS:", np.where(target_au > 0)[0])
print("Birds: ", [id2bird[el] for el in np.where(target_au > 0)[0]])

# plt.figure(figsize=(20,10))
spec = mel_spec(torch.from_numpy((sample_au))[None,...]) #** 0.6080866059834527
plt.imshow(spec[0])
plt.show()

ipd.Audio(sample_au, rate=32000)

In [ ]:
(spec - spec_new).abs().amax(axis=(1,2))

In [ ]:
target_au

In [ ]:
sample_au_full, sample_sr_full = librosa.load(
    train_dataset.df["filename_with_root"].iloc[sample_id].replace("train_features", "train_audio").replace(".hdf5", ".ogg"),
    sr=None
)
plt.imshow(mel_spec(torch.from_numpy(sample_au_full[2097528:2097528 + 5*32000])[None,...])[0])

In [ ]:
np.allclose(sample_au, librosa.util.normalize(sample_au_full[2097528:2097528 + 5*32000]))

In [ ]:
sample_au.shape

In [ ]:
sample_au[:sample_sr*,]

In [ ]:
np.abs(sample_au).max()

In [ ]:
ipd.Audio(sample_au, rate=32000)

In [ ]:
# sample_au, target_au = train_dataset[0]
sample_au, target_au = train_dataset[35706]

plt.figure(figsize=(20,10))
spec = mel_spec(torch.from_numpy(sample_au)[None,...])
plt.imshow(spec[0])
plt.show()

In [ ]:
ipd.Audio(sample_au, rate=32000)

In [ ]:
check_labels = torch.stack([train_dataset_full[idx][1] for idx in range(990,990+30)])

In [ ]:
torch.where(check_labels > 0)[1]

In [ ]:
train_dataset_full.df.iloc[120]

In [ ]:
train_dataset_full.sampleidx_2_dfidx[991]

In [ ]:
original_sample = librosa.load("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_audio/asbfly/XC134896.ogg", sr=32000)[0]

In [ ]:
librosa.util.normalize(original_sample[:32000*5])

In [ ]:
full_sample_au, full_target_au, dfidx, start, end = train_dataset_full[1]

In [ ]:
np.allclose(full_sample_au, librosa.util.normalize(original_sample[32000*5:32000*10]))

In [ ]:
ipd.Audio(sample_au_aug, rate=32000)

In [ ]:
target_au_aug

In [ ]:
maybe_dupl1 = librosa.util.normalize(librosa.load(
    "/home/vova/data/exps/birdclef_2024/birdclef_2022/train_audio/brant/XC294370.ogg", sr=32000
)[0])
maybe_dupl2 = librosa.util.normalize(librosa.load(
    "/home/vova/data/exps/birdclef_2024/birdclef_2022/train_audio/gadwal/XC294370.ogg", sr=32000
)[0])

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(mel_spec(torch.from_numpy(maybe_dupl1)[None,...])[0])
plt.show()

plt.figure(figsize=(20,10))
plt.imshow(mel_spec(torch.from_numpy(maybe_dupl2)[None,...])[0])
plt.show()

In [ ]:
# ipd.Audio(sample_au, rate=32_000)

In [ ]:
ipd.Audio(sample_au_aug, rate=32_000)

In [ ]:
np.abs(sample_au_aug).max()

In [ ]:
torch.unique(target_au_aug), torch.sum(target_au_aug)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(my_spec(torch.from_numpy(sample_au_aug)[None,:])[0])
plt.show();

In [ ]:
target_au.max()

In [ ]:
torch.allclose(target_au, target_au_aug)

In [ ]:
np.abs(sample_au - sample_au_aug).max()

In [ ]:
plt.plot(sample_au)

In [ ]:
plt.plot(sample_au_aug)

In [ ]:
ipd.Audio(sample_au, rate=32000)

In [ ]:
ipd.Audio(sample_au_aug, rate=32000)

In [ ]:
np.allclose(sample_au, sample_feat)

In [ ]:
# train_dataset = WaveDataset(
#     df=df,
#     root="/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/train_audio/",
#     label_str2int_mapping_path="/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/bird2int_2023.json",
#     precompute=False,
#     n_cores=32,
#     debug=True,
#     do_mixup=False,
#     # mixup_params={"prob": 0.5, "alpha": None},
#     segment_len=5.0,
#     late_normalize=True,
#     sec_target_col=None
# )

# train_dataset_extended = WaveDataset(
#     df=df,
#     root="/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/train_audio/",
#     label_str2int_mapping_path="/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/bird2int_2023.json",
#     precompute=True,
#     n_cores=32,
#     debug=False,
#     do_mixup=True,
#     mixup_params={"prob": 0.5, "alpha": None},
#     segment_len=5.0,
#     late_normalize=True,
#     add_df_paths=[
#         "/home/vova/data/exps/BirdCLEF_2023/train_metadata_extended_2020_2022_no2023_scored.csv"
#     ],
#     sampler_col="sw_col"
# )

In [ ]:
train_dataset[11][1].sum()

In [ ]:
train_dataset.df[train_dataset.df.secondary_labels.apply(len) > 1]

In [ ]:
train_dataset_extended.df[]

In [ ]:
set(train_dataset_extended.df["sw_col"]) - set(load_json("/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/scored_birds.json"))

In [ ]:
train_dataset_extended.df["filename_with_root"].iloc[-1]

In [ ]:
sample_au, sample_label = train_dataset[0]
sample_au = torch.from_numpy(sample_au)[None,:]

In [ ]:
sample_au_spec = model.forward(sample_au, return_spec_feature=True)

In [ ]:
plt.imshow(sample_au_spec[0].numpy())

In [ ]:
# ipd.Audio(sample_au, rate=32_000)

In [ ]:
aug_block = RandomFiltering(
    min_db=-20,
    is_wave=True,
    normalize_wave=True
)

In [ ]:
auged_sample_au = aug_block(sample_au)

In [ ]:
auged_sample_au_spec = model.forward(auged_sample_au, return_spec_feature=True)

In [ ]:
plt.imshow(auged_sample_au_spec[0].numpy())

In [ ]:
plt.imshow(auged_sample_au_spec[0].numpy() - sample_au_spec[0].numpy())

In [ ]:
ipd.Audio(auged_sample_au, rate=32_000)

In [ ]:
valid_dataset = WaveAllFileDataset(
    df=df,
    root="/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/train_audio/",
    label_str2int_mapping_path="/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/bird2int_2023.json",
    precompute=True,
    n_cores=32,
    debug=True,
    segment_len=5,
    lookback=2.5,
    lookahead=2.5,
    late_normalize=True,
    sample_id=None
)

In [ ]:
sample_au, sample_label, sample_df_id, sample_start, sample_end_s = valid_dataset[0]

In [ ]:
len(sample_au) / 32_000

In [ ]:
sample_au

# Sounscapes

In [ ]:
un_ss = glob("/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes/*.ogg")

In [ ]:
pseudo = pd.read_csv("../pseudo/v1/positive_filtered_07.csv", converters={"all_labels": eval})

In [ ]:
pseudo

In [ ]:
un_ss

In [ ]:
pseudo["sample_id"] = pseudo["row_id"].apply(lambda x: x.split("_")[0])

In [ ]:
pseudo

In [ ]:
pseudo[pseudo["sample_id"] == "100238396"]

In [ ]:
pseudo_samples

In [ ]:
df_idx = 6143

In [ ]:
sample_idx, end_second = pseudo["row_id"].iloc[df_idx].split("_")

sample, _ = librosa.load(
    # un_ss[500], 
    f'/home/vova/data/exps/birdclef_2024/birdclef_2024/unlabeled_soundscapes/{sample_idx}.ogg',
    sr=None
)
# sample = librosa.util.normalize(sample)

In [ ]:
sample_slice = sample[32000*(int(end_second) - 5):32000*int(end_second)]
ipd.Audio(sample_slice, rate=32000)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(mel_spec(torch.from_numpy(sample_slice)[None,...])[0])
plt.show()

# Compare librosa and torchaudio

In [ ]:
librosa_loaded = librosa.load("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_audio/asbfly/XC453302.ogg", sr=32000)[0]

In [ ]:
torchaudio_loaded, torchaudio_sr = torchaudio.load("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_audio/asbfly/XC453302.ogg")
torchaudio_loaded = Resample(orig_freq=torchaudio_sr, new_freq=16000)(torchaudio_loaded)[0]

In [ ]:
torchaudio_loaded.shape

In [ ]:
librosa_loaded.shape

In [ ]:
len(librosa_loaded) / 32000

In [ ]:
len(torchaudio_loaded) / 32000

In [ ]:
np.abs(torchaudio_loaded[len(torchaudio_loaded) - len(librosa_loaded):] - librosa_loaded).max()

In [ ]:
np.abs(torchaudio_loaded[:len(librosa_loaded)] - librosa_loaded).max()

In [ ]:
np.allclose(torchaudio_loaded[:len(librosa_loaded)], librosa_loaded)

In [ ]:
len(librosa_loaded) / 32000

In [ ]:
len(torchaudio_loaded) / 32000

# Spectogram

In [ ]:
test_pred = pd.read_csv("submission.csv")

In [ ]:
np.sort(test_pred.iloc[:,1:].values.flatten())[-20:]

In [ ]:
np.where(test_pred.iloc[:,1:].max(axis=1) > 0.9)

In [ ]:
test_pred.iloc[65]

In [ ]:
test_audio, test_sr = librosa.load("/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/test_soundscapes/soundscape_29201.ogg", sr=None)
test_audio = librosa.util.normalize(test_audio)

In [ ]:
test_sr

In [ ]:
ipd.Audio(
    test_audio[test_sr*0:test_sr*5],
    rate=test_sr
)

In [ ]:
test_sr

In [ ]:
test_pred.iloc[65].iloc[1:].sort_values()

In [ ]:
mel_spec_paramms={
    "sample_rate": 32000,
    "n_mels": 128,
    "f_min": 20,
    "n_fft": 2048,
    "hop_length": 512,
    "normalized": True,
}
mel_spec = nn.Sequential(
    TraceableMelspec(**mel_spec_paramms, quantizable=False),
    AmplitudeToDB(top_db=80.0),
    NormalizeMelSpec(exportable=True),
)

# leaf_spec = {
#     "sample_rate": 32000,
#     "n_filters": 128,
#     # "window_stride": 15
# }

# leaf_complex = nn.Sequential(
#     frontend.Leaf(**leaf_spec, onnx_export=True, compression_fn=None),
#     AmplitudeToDB(top_db=80.0),
#     NormalizeMelSpec(exportable=True),
# )

# leaf_noncomplex = nn.Sequential(
#     frontend.Leaf(**leaf_spec, onnx_export=True),
#     NormalizeMelSpec(exportable=True),
# )

In [ ]:
spec = mel_spec(torch.from_numpy(librosa.util.normalize(test_audio[
    test_sr*0:
    test_sr*5
]))[None,:])
spec = torch.cat([spec,spec,spec])

# leaf_1 = leaf_complex(torch.from_numpy(librosa.util.normalize(test_audio[
#     test_sr*0:
#     test_sr*5
# ]))[None,:])
# leaf_2 = leaf_noncomplex(torch.from_numpy(librosa.util.normalize(test_audio[
#     test_sr*0:
#     test_sr*5
# ]))[None,:])

In [ ]:
class CustomMasking(nn.Module):
    def __init__(
        self,
        mask_max_length: int,
        mask_max_masks: int,
        p=1.0,
        inplace=True
    ):
        super().__init__()
        assert isinstance(mask_max_masks, int) and mask_max_masks > 0 
        self.mask_max_masks = mask_max_masks
        self.mask_max_length = mask_max_length
        self.mask_module = None
        self.p = p
        self.inplace = inplace
    
    def forward(self, x):
        if not self.inplace:
            output = x.clone()
        for i in range(x.shape[0]):
            if np.random.binomial(n=1, p=self.p):
                n_applies = np.random.randint(low=1, high=self.mask_max_masks+1)
                print(f"Applying {n_applies} times")
                for _ in range(n_applies):
                    if self.inplace:
                        x[i:i+1] = self.mask_module(x[i:i+1])
                    else:
                        output[i:i+1] = self.mask_module(output[i:i+1])
        if self.inplace:
            return x
        else: 
            return output
        
class CustomTimeMasking(CustomMasking):
    def __init__(
        self,
        mask_max_length: int,
        mask_max_masks: int,
        p=1.0,
        inplace=True
    ):
        super().__init__(
            mask_max_length=mask_max_length,
            mask_max_masks=mask_max_masks,
            p=p,
            inplace=inplace
        )
        self.mask_module = TimeMasking(
            time_mask_param=mask_max_length
        )
        
class CustomFreqMasking(CustomMasking):
    def __init__(
        self,
        mask_max_length: int,
        mask_max_masks: int,
        p=1.0,
        inplace=True
    ):
        super().__init__(
            mask_max_length=mask_max_length,
            mask_max_masks=mask_max_masks,
            p=p,
            inplace=inplace
        )
        self.mask_module = FrequencyMasking(
            freq_mask_param=mask_max_length
        )

In [ ]:
np.random.randint(low=1, high=1+1)

In [ ]:
# time_masking = CustomTimeMasking(
#     mask_max_length=20,
#     mask_max_masks=3,
#     p=0.5,
#     inplace=True
# )
# freq_masking = CustomFreqMasking(
#     mask_max_length=10,
#     mask_max_masks=3,
#     p=0.5,
#     inplace=True
# )

In [ ]:
both_masking = nn.Sequential(
    CustomTimeMasking(
        mask_max_length=20,
        mask_max_masks=3,
        p=0.3,
        inplace=True
    ),
    CustomFreqMasking(
        mask_max_length=10,
        mask_max_masks=3,
        p=0.3,
        inplace=True
    )
)

In [ ]:
# time_masked = time_masking(spec)
# freq_masked = freq_masking(spec)

both_masked = both_masking(spec)

In [ ]:
swin = timm.create_model(
    "swinv2_tiny_window8_256", 
    in_chans=1, 
    # features_only=True,
    dropout_rate=0.2,
    num_classes=20
)
# swin.head.drop.p = 0.4
# swin.head = nn.Identity()

In [ ]:
swin.head

In [ ]:
swin.train();

In [ ]:
swin.head.drop.p = 0.4

In [ ]:
swin(spec[:,None,:,:])[-1]

In [ ]:
torch.abs(leaf_1- leaf_2).max()

In [ ]:
time_masked = time_masking(spec[0])
freq_masked = freq_masking(spec[0])

In [ ]:
# time_masked = time_masking(spec)
# freq_masked = freq_masking(spec)

plt.figure(figsize=(10,10))
plt.imshow(spec[0].detach().cpu().numpy())
plt.show();

plt.figure(figsize=(10,10))
plt.imshow(spec[1].detach().cpu().numpy())
plt.show();

plt.figure(figsize=(10,10))
plt.imshow(spec[2].detach().cpu().numpy())
plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(both_masked[0].detach().cpu().numpy())
# plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(both_masked[1].detach().cpu().numpy())
# plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(both_masked[2].detach().cpu().numpy())
# plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(freq_masked[0].detach().cpu().numpy())
# plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(freq_masked[1].detach().cpu().numpy())
# plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(freq_masked[2].detach().cpu().numpy())
# plt.show();

# plt.figure(figsize=(10,10))
# plt.imshow(freq_masked[0].detach().cpu().numpy())
# plt.show();

In [ ]:
test_audio, test_sr = librosa.load("/home/vova/data/exps/BirdCLEF_2023/birdclef_2023/train_audio/pygbat1/XC755905.ogg", sr=None)
test_audio = librosa.util.normalize(test_audio)

In [ ]:
test_sr

In [ ]:
spec = my_spec(torch.from_numpy(librosa.util.normalize(test_audio))[None,:])
# cqt_spec = cqt(torch.from_numpy(sample_au)[None,:])

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(spec[0])
plt.show();
# plt.imshow(cqt_spec[0])
# plt.show();

In [ ]:
spec.shape, cqt_spec.shape

In [ ]:
spec.max(), spec.min()

In [ ]:
cqt_spec.max(), cqt_spec.min()

# Tune Scheduler

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau, OneCycleLR

In [ ]:
def visualise_lr_scheduling(lr_scheduler_from_opt, n_steps, verbose=True, lr=1e-3):
    # t_opt = torch.optim.SGD([torch.tensor(1)], lr=lr)
    t_opt = torch.optim.SGD([
        {'params': torch.tensor(1), 'lr': lr},
        # {'params': torch.tensor(1), 'lr': lr/20}
    ])
    t_sched = lr_scheduler_from_opt(t_opt)
    lrs = []
    for i in range(n_steps):
        t_opt.step()
        lrs.append(t_sched.get_lr()[0])
        t_sched.step()
        
    if verbose:
        plt.title("Learning Rate Schedule")
        plt.xlabel("Iteration")
        plt.ylabel("Learning Rate")
        plt.plot(lrs)
        plt.show()
        
    print(f"Min LR : {min(lrs)}. Max LR: {max(lrs)}")
    return lrs

In [ ]:
visualise_lr_scheduling(
    lr_scheduler_from_opt=lambda opt: GradualWarmupSchedulerV2(
        optimizer=opt, 
        multiplier=10,
        total_epoch=5,
        after_scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
            opt,
            T_max=45,
            eta_min=1e-7
        )
    ),
    n_steps=50+2,
    lr=1e-5
);

# Model

In [ ]:
timm.list_models("tf_efficientnetv2*")

In [ ]:
backbone_model = timm.create_model("efficientformerv2_s2.snap_dist_in1k", features_only=True)

In [ ]:
backbone_model

In [ ]:
TRAIN_PERIOD = 5.0

# model_config = dict(
#     backbone="eca_nfnet_l0",
#     add_backbone_config={
#         "drop_path_rate": 0.2
#     },
#     mel_spec_paramms={
#         "sample_rate": 32000,
#         "n_mels": 128,
#         "f_min": 20,
#         "n_fft": 2048,
#         "hop_length": 512,
#         "normalized": True,
#     },
#     head_config={
#         "p": 0.5,
#         "num_class": 264,
#         "train_period": TRAIN_PERIOD,
#         "infer_period": TRAIN_PERIOD,
#     },
#     exportable=True,
#     # deep_supervision_steps=3
# )
model_config = dict(
    backbone="tf_efficientnetv2_b1.in1k",
    mel_spec_paramms={
        "sample_rate": 32000,
        "n_mels": 128,
        "f_min": 20,
        "n_fft": 2048,
        "hop_length": 512,
        "normalized": True,
    },
    spec_augment_config={
        "bandpass_noise":{
            "p": 1.0,
            "inplace": True
        },
        # "power_aug":{
        #     "power_range": (0.5, 3.0),
        #     "p": 1.0,
        #     "inplace": True
        # },
        # "freq_mask": {
        #     "mask_max_length": 20,
        #     "mask_max_masks": 3,
        #     "p": 0.7,
        #     "inplace": True,
        # },
        # "time_mask": {
        #     "mask_max_length": 30,
        #     "mask_max_masks": 3,
        #     "p": 0.7,
        #     "inplace": True,
        # },
    },
    head_config={
        "p": 0.5,
        "num_class": 998,
        "train_period": TRAIN_PERIOD,
        "infer_period": TRAIN_PERIOD,
    },
    exportable=True,
    fixed_amplitude_to_db=True
)

model = WaveCNNAttenClasifier(**model_config, device="cpu")

model.train();

In [ ]:
spec = model.forward(torch.from_numpy(sample_au)[None,...], return_spec_feature=True)
plt.imshow(spec[0,0])

In [ ]:
plt.hist(noise.flatten(), bins=30)

In [ ]:
plt.hist(np.random.uniform(low=0.10, high=0.2, size=(10000,)))

In [ ]:
import random

In [ ]:
random.uniform(0.2, 0.3)

In [ ]:
# noise = (np.random.sample((spec[0,0].shape[0], spec[0,0].shape[1])).astype(np.float32)) * 0.05 * (np.random.sample() + 0.3)
plt.imshow(spec[0,0] + torch.randn_like(spec[0,0]) * 0.2 * (np.random.sample() + 0.3))

In [ ]:
torch.randn_like(spec[0,0])

In [ ]:
plt.hist(torch.randn_like(spec[0,0]).flatten(), bins=30)

In [ ]:
plt.imshow(spec[0,0])

In [ ]:
plt.imshow(spec[0,0])

In [ ]:
plt.imshow(spec[0,0])

In [ ]:
plt.imshow(spec[0,0]* pink_noise)

In [ ]:
import random

In [ ]:
noise = (np.random.sample((spec[0,0].shape[0], spec[0,0].shape[1])).astype(np.float32)) * 0.05 * (np.random.sample() + 0.3)

In [ ]:
r = random.randint(spec[0,0].shape[0]//2,spec[0,0].shape[0])
x = random.random()/2
pink_noise = torch.from_numpy(np.array([np.concatenate((1-np.arange(r)*x/r,np.zeros(spec[0,0].shape[0]-r)-x+1))]).T)

In [ ]:
pink_noise.shape

In [ ]:
spec[0,0] = spec[0,0] * pink_noise 

In [ ]:
plt.hist(torch.randn_like(spec[0,0]).numpy().flatten())

In [ ]:
spec[0,0].min()

In [ ]:
sample = torch.randn(4, int(32000 * TRAIN_PERIOD))

In [ ]:
with torch.no_grad():
    out = model(sample)

In [ ]:
out["clipwise_pred_long"]

In [ ]:
out_2["clipwise_pred_long"]

In [ ]:
out_old["clipwise_pred_long"]

In [ ]:
out_old_2["clipwise_pred_long"]

In [ ]:
both_masking = nn.Sequential(
    CustomTimeMasking(
        mask_max_length=20,
        mask_max_masks=3,
        p=0.3,
        inplace=True
    ),
    CustomFreqMasking(
        mask_max_length=10,
        mask_max_masks=3,
        p=0.3,
        inplace=True
    )
)

In [ ]:
128 / 8

In [ ]:
256 / 8

In [ ]:
model

In [ ]:
sample_au = torch.from_numpy(sample_au)[None,...]
# sample_au = torch.stack([sample_au, sample_au, sample_au, sample_au])

In [ ]:
spec = model.forward(
    sample_au,
    # torch.from_numpy(sample_au)[None,...],
    return_spec_feature=True
)

In [ ]:
spec

In [ ]:
len(spec)

In [ ]:
plt.imshow(spec[0,0].detach())

In [ ]:
plt.imshow(spec[0,0].detach())

In [ ]:
some_backbone = timm.create_model(
    "eca_nfnet_l0",
    features_only=True,
    pretrained=True,
    exportable=True,
    in_chans=2,
    out_indices=None
)
some_backbone.eval();

In [ ]:
some_backbone.feature_info.info

In [ ]:
sample_input = torch.randn((1,2,256,256))

In [ ]:
sample_input.mean((2,3), keepdims=True)

In [ ]:
type(sample_output_1)

In [ ]:
with torch.no_grad():
    sample_output_2 = some_backbone(sample_input)

In [ ]:
len(sample_output_1), len(sample_output_2)

In [ ]:
torch.allclose(sample_output_1[3], sample_output_2[1])

In [ ]:
sample = torch.randn((4, int(TRAIN_PERIOD*32_000)))

In [ ]:
model(sample)["clipwise_pred"].shape

In [ ]:
sample = torch.from_numpy(librosa.util.normalize(test_audio[
    test_sr*0:
    test_sr*5
]))[None,:]

In [ ]:
sample.shape

In [ ]:
spec = model.logmelspec_extractor(sample)

In [ ]:
spec.shape

In [ ]:
spec.shape

In [ ]:
model.logmelspec_extractor

In [ ]:
embs = model.backbone(spec)

In [ ]:
# model.train();
model.eval();
embs = model.forward(sample, return_spec_feature=True)

In [ ]:
plt.imshow(embs[0,0])

In [ ]:
embs["clipwise_pred_long"].min()

In [ ]:
embs.shape

In [ ]:
len(embs)

In [ ]:
model.forward(sample)

In [ ]:
embs.transpose()

In [ ]:
# torch.Size([4, 2304, 4, 10])

In [ ]:
len(embs)

In [ ]:
probs = model.forward(sample)

In [ ]:
probs

In [ ]:
embs[2].shape

In [ ]:
pool_layers = nn.ModuleList([
    GeMFreqFixed(time_kernel_size=k*2 if k>0 else 1) for k in list(reversed(range(len(embs))))
])

In [ ]:
pooled_embs = [p(emb) for p, emb in zip(pool_layers, embs)]

In [ ]:
for i in range(len(embs)):
    print(pooled_embs[i].shape, embs[i].shape)

In [ ]:
pooled_embs[3].shape, embs[3].shape

In [ ]:
[k*2 if k>0 else 1 for k in list(reversed(range(len(embs))))]

In [ ]:
list(reversed(range(1, len(embs))))

In [ ]:
[el.shape for el in embs]

In [ ]:
avg_pool = torch.nn.AvgPool2d(kernel_size=(2,2))

In [ ]:
avg_pool(embs[-2]).shape

In [ ]:
spec = model(sample)

In [ ]:
spec.shape

In [ ]:
!rm test_onnx -rf

In [ ]:
model.eval()
convert_to_onnx(
    model_to_convert=model,
    sample_input=torch.randn(4, int(TRAIN_PERIOD * 32_000)),
    # base_path=f"../logdirs/tf_efficientnetv2_m_in21ft1k_convnext_tiny_in22ft1k_eca_nfnet_l0_noval_v3AllTune_075Clipwise025TimeMax_GausMean/onnx_ensem"
    base_path="test_onnx"
)

In [ ]:
model = ort.InferenceSession(f"test_onnx/model_simpl.onnx")

In [ ]:
out = model.run(None, {"input": torch.randn(4, int(TRAIN_PERIOD * 32_000)).numpy()})[0]

In [ ]:
out.shape

In [ ]:
out

# Losses

In [ ]:
from code_base.losses.focal_loss import FocalLossBCE

In [ ]:
loss_f = FocalLossBCE(reduction="None")

In [ ]:
((torch.rand(100000) < 0.8).float()).mean()

In [ ]:
inputs = torch.randn()

In [ ]:
loss_f.forward()

# Prepare final datasets

In [ ]:
train_metadata_extended_202x = pd.read_csv(
    "/home/vova/data/exps/BirdCLEF_2023/train_metadata_extended_2020_2022_no2023_scored_nodupl_v1_only_prime_2023SecLabels.csv"
)
xc_add_first = pd.read_csv(
    "/home/vova/data/exps/BirdCLEF_2023/xeno_canto/scored_2023_xc_2023SecLabels.csv"
)
xc_add_second = pd.read_csv(
    "/home/vova/data/exps/BirdCLEF_2023/xeno_canto/scored_2023_xc_2023SecLabels_till_12_05_2023_v2.csv"
)

In [ ]:
xc_add_first.filename.iloc[0]

In [ ]:
for fname in tqdm(xc_add_second["filename"]):
    splitted_fname = fname.split("/")
    new_fname = os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/audio/", splitted_fname[-2], splitted_fname[-1])
    if not os.path.exists(os.path.dirname(new_fname)):
        os.makedirs(os.path.dirname(new_fname))
    copyfile(
        fname,
        os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/audio/", splitted_fname[-2], splitted_fname[-1])
    )

In [ ]:
def modify_name(input):
    splitted_fname = input.split("/")
    return os.path.join(splitted_fname[-2], splitted_fname[-1])

In [ ]:
train_metadata_extended_202x["filename"] = train_metadata_extended_202x["filename"].apply(modify_name)
xc_add_first["filename"] = xc_add_first["filename"].apply(modify_name)
xc_add_second["filename"] = xc_add_second["filename"].apply(modify_name)

In [ ]:
train_metadata_extended_202x.drop(columns="Unnamed: 0").to_csv(
    "/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/train_metadata_extended_2020_2022_no2023_scored_nodupl_v1_only_prime_2023SecLabels.csv",
    index=False
)
xc_add_first.to_csv(
    "/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/scored_2023_xc_2023SecLabels.csv",
    index=False
)
xc_add_second.to_csv(
    "/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/scored_2023_xc_2023SecLabels_till_12_05_2023_v2.csv",
    index=False
)

In [ ]:
background_noise = pd.read_csv(
    "/home/vova/data/exps/BirdCLEF_2023/soundscapes_processed/v1_no_call_meta.csv"
)

In [ ]:
background_noise["filename"].iloc[0]

In [ ]:
for fname in tqdm(background_noise["filename"]):
    splitted_fname = fname.split("/")
    new_fname = os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/soundscapes_nocall/", splitted_fname[-2], splitted_fname[-1])
    if os.path.exists(new_fname):
        continue
    if not os.path.exists(os.path.dirname(new_fname)):
        os.makedirs(os.path.dirname(new_fname))
    copyfile(
        fname,
        new_fname
    )

In [ ]:
background_noise["filename"] = background_noise["filename"].apply(os.path.basename)

In [ ]:
background_noise

In [ ]:
background_noise.to_csv(
    "/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/v1_no_call_meta.csv",
    index=False
)

In [ ]:
background_noise_esc50 = pd.read_csv(
    "../../BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/esc50_background.csv"
)

In [ ]:
for fname in tqdm(background_noise_esc50["filename"]):
    splitted_fname = fname.split("/")
    new_fname = os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/esc50/", splitted_fname[-2], splitted_fname[-1])
    if os.path.exists(new_fname):
        continue
    if not os.path.exists(os.path.dirname(new_fname)):
        os.makedirs(os.path.dirname(new_fname))
    copyfile(
        fname,
        new_fname
    )

In [ ]:
background_noise_esc50["filename"] = background_noise_esc50["filename"].apply(os.path.basename)

In [ ]:
background_noise_esc50.to_csv(
    "/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part1/esc50_background.csv",
    index=False
)

In [ ]:
pretrain_dataset = pd.read_csv(
    "/home/vova/data/exps/BirdCLEF_2023/xeno_canto/train_metadata_extended_shorten_and_2023x_v3.csv"
)

In [ ]:
pretrain_dataset["filename"].apply(lambda x: "/".join(x.split("/")[:-2])).value_counts()

In [ ]:
pretrain_dataset_2021 = pretrain_dataset[
    pretrain_dataset["filename"].apply(lambda x: "/".join(x.split("/")[:-2])) == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2021/train_short_audio"
]

In [ ]:
pretrain_dataset_2021["filename"].apply(lambda x: "/".join(x.split("/")[:-2])).value_counts()

In [ ]:
pretrain_dataset_2021["filename"].iloc[0]

In [ ]:
for fname in tqdm(pretrain_dataset_2021["filename"]):
    splitted_fname = fname.split("/")
    new_fname = os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part5/birdclef_2021", splitted_fname[-2], splitted_fname[-1])
    if os.path.exists(new_fname):
        continue
    if not os.path.exists(os.path.dirname(new_fname)):
        os.makedirs(os.path.dirname(new_fname))
    copyfile(
        fname,
        new_fname
    )

In [ ]:
pretrain_dataset_xc = pretrain_dataset[
    pretrain_dataset["filename"].apply(lambda x: "/".join(x.split("/")[:-2])) == "/home/vova/data/exps/BirdCLEF_2023/xeno_canto/dataset/audio"
]

In [ ]:
pretrain_dataset_xc["filename"].apply(lambda x: "/".join(x.split("/")[:-2])).value_counts()

In [ ]:
for fname in tqdm(pretrain_dataset_xc["filename"]):
    splitted_fname = fname.split("/")
    new_fname = os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part6/xeno_canto/", splitted_fname[-2], splitted_fname[-1])
    if os.path.exists(new_fname):
        continue
    if not os.path.exists(os.path.dirname(new_fname)):
        os.makedirs(os.path.dirname(new_fname))
    copyfile(
        fname,
        new_fname
    )

In [ ]:
pretrain_dataset["temp"] = pretrain_dataset["filename_h5py"].apply(lambda x: "/".join(x.split("/")[:-2]))

In [ ]:
pretrain_dataset_2022 = pretrain_dataset[
    pretrain_dataset["filename"].apply(lambda x: "/".join(x.split("/")[:-2])) == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2022/train_audio"
]

In [ ]:
pretrain_dataset["temp"].value_counts()

In [ ]:
for fname in tqdm(pretrain_dataset_2022["filename"]):
    splitted_fname = fname.split("/")
    new_fname = os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/birdclef_2023_data_part8/birdclef_2022", splitted_fname[-2], splitted_fname[-1])
    if os.path.exists(new_fname):
        continue
    if not os.path.exists(os.path.dirname(new_fname)):
        os.makedirs(os.path.dirname(new_fname))
    copyfile(
        fname,
        new_fname
    )

In [ ]:
# /home/vova/data/exps/BirdCLEF_2023/birdclef_2020/audio_features
pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2020/audio_features",
    "filename_h5py"
] = pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2020/audio_features",
    "filename_h5py"
].apply(lambda x: x.replace("/home/vova/data/exps/BirdCLEF_2023/birdclef_2020/audio_features", "data/birdclef_2020_features"))
# /home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_a_m/audio_features
pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_a_m/audio_features",
    "filename_h5py"
] = pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_a_m/audio_features",
    "filename_h5py"
].apply(lambda x: x.replace("/home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_a_m/audio_features", "data/birdclef_2020_xc_a_m_features"))
# /home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_n_z/audio_features
pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_n_z/audio_features",
    "filename_h5py"
] = pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_n_z/audio_features",
    "filename_h5py"
].apply(lambda x: x.replace("/home/vova/data/exps/BirdCLEF_2023/birdclef_2020_xc_n_z/audio_features", "data/birdclef_2020_xc_n_z_features"))
# /home/vova/data/exps/BirdCLEF_2023/birdclef_2021/audio_features
pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2021/audio_features",
    "filename_h5py"
] = pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2021/audio_features",
    "filename_h5py"
].apply(lambda x: x.replace("/home/vova/data/exps/BirdCLEF_2023/birdclef_2021/audio_features", "data/birdclef_2021_features"))
# /home/vova/data/exps/BirdCLEF_2023/birdclef_2022/train_audio
pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2022/audio_features",
    "filename_h5py"
] = pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/birdclef_2022/audio_features",
    "filename_h5py"
].apply(lambda x: x.replace("/home/vova/data/exps/BirdCLEF_2023/birdclef_2022/audio_features", "data/birdclef_2022_features"))
# /home/vova/data/exps/BirdCLEF_2023/xeno_canto/audio_features
pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/xeno_canto/audio_features",
    "filename_h5py"
] = pretrain_dataset.loc[
    pretrain_dataset["temp"] == "/home/vova/data/exps/BirdCLEF_2023/xeno_canto/audio_features",
    "filename_h5py"
].apply(lambda x: x.replace("/home/vova/data/exps/BirdCLEF_2023/xeno_canto/audio_features", "data/xeno_canto_features"))

In [ ]:
pretrain_dataset.loc[
    pretrain_dataset["filename_h5py"].apply(lambda x: os.path.exists(os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place", x))), 
    "temp"
].value_counts()

In [ ]:
pretrain_dataset.loc[
    ~pretrain_dataset["filename_h5py"].apply(lambda x: os.path.exists(os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place", x))), 
    "temp"
].value_counts()

In [ ]:
pretrain_dataset = pretrain_dataset.drop(columns=["temp"])
pretrain_dataset["filename"] = pretrain_dataset["filename"].apply(modify_name)

In [ ]:
pretrain_dataset.to_csv(
    "/home/vova/src/exps/BirdCLEF_2023_1st_place/data/train_metadata_extended_shorten_and_2023x_v3.csv",
    index=False
)

In [ ]:
background_noise_esc50

In [ ]:
v1_no_call_meta = pd.read_csv("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/v1_no_call_meta.csv")

In [ ]:
v1_no_call_meta[
    ~v1_no_call_meta["filename"].apply(lambda x: os.path.exists(os.path.join("/home/vova/src/exps/BirdCLEF_2023_1st_place/data/soundscapes_nocall/train_audio/", x)))
]